## Data prep

In [5]:
import pandas as pd
df = pd.read_csv("data/songdata.zip")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
vocab = set()
for song in df["text"]:
    chars = set(song)
    vocab = vocab.union(chars)
vocab_size = len(vocab)
print("Vocab size:", len(vocab))

Vocab size: 76


In [7]:
chars = list(vocab)
char2idx = { char:i for i,char in enumerate(chars) }
idx2char = { i:char for i,char in enumerate(chars) }

In [8]:
def build_samples(song, buffer_length):
    tokens = song

    x_train = []
    y_train = []
    for i in range(0, len(song)):
        if i+buffer_length+1 >= len(tokens):
            continue
            
        x_train.append(tokens[i:i+buffer_length])
        y_train.append(tokens[i+buffer_length])

    return x_train,y_train

In [9]:
x_train, y_train = [], []
for song in df["text"][:100]:
    xs, ys = build_samples(song, 6)
    x_train.extend(xs)
    y_train.extend(ys)
print("Training data length:", len(x_train))
print("X[0]:", x_train[0])
print("Y[0]:", y_train[0])

Training data length: 133204
X[0]: Look a
Y[0]: t


## Train/Test split

In [10]:
from sklearn.model_selection import train_test_split
import math

# 80% Train, 10% Dev, 10% Test
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

In [11]:
import numpy as np
from random import shuffle
import util

SEQUENCE_LENGTH = 6

def generate_batches(data_length, mini_batch_size):
    for begin in range(0, data_length, mini_batch_size):
        end = min(begin + mini_batch_size, data_length)
        yield begin, end

def load_batch(xs, ys, begin, end):
    batch_size = end-begin
    
    x_train = np.zeros((batch_size, SEQUENCE_LENGTH, vocab_size))
    y_train = np.zeros((batch_size, vocab_size))
    
    xs_batch = xs[begin:end]
    ys_batch = ys[begin:end]
    
    c = list(zip(xs_batch, ys_batch))
    shuffle(c)
    xs_batch, ys_batch = zip(*c)
    
    for i in range(batch_size):
        x_train[i] = util.one_hot_encode_sequence(xs_batch[i], char2idx)
        y_train[i] = util.one_hot_encode(ys_batch[i], char2idx)
    
    return x_train, y_train

In [12]:
batches = generate_batches(len(X_train), 512)
begin, end = next(batches)
x_batch, y_batch = load_batch(X_train, Y_train, begin, end)
print(x_batch.shape, y_batch.shape)

(512, 6, 76) (512, 76)


In [13]:
for i in range(10):
    xs = ''.join(util.one_hot_decode_sequence(x_batch[i], idx2char))
    y = util.one_hot_decode(y_batch[i], idx2char)
    print(f"{i} '{xs}' -> {y}")

0 'rted  ' -> 

1 'n  
It' -> '
2 ' 
Sorr' -> y
3 ' calls' ->  
4 ' 
  
Y' -> o
5 'have a' ->  
6 'ound  ' -> 

7 'rilyn ' -> F
8 'e  
Do' ->  
9 'find m' -> e


## Training time!

In [17]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, GRU
from keras.layers import LeakyReLU
from pathlib import Path

def build_model(vocab_size):
    model = Sequential()
    model.add(LSTM(512, input_shape=(SEQUENCE_LENGTH, vocab_size)))
    model.add(Dropout(0.5))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer="adam", metrics = ['accuracy'])
    return model

model = build_model(vocab_size)
weights_path = Path("weights_char_rnn.h5")
if weights_path.is_file():
    print("Loaded!")
    model.load_weights(weights_path.resolve())
model.summary()

Loaded!
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 512)               1206272   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 76)                38988     
Total params: 1,245,260
Trainable params: 1,245,260
Non-trainable params: 0
_________________________________________________________________


In [90]:
batches = generate_batches(len(X_train), 4096)

In [91]:
for begin, end in batches:
    x_batch, y_batch = load_batch(X_train, Y_train, begin, end)
    model.fit(x_batch, y_batch, batch_size=256, epochs=4)

Epoch 1/4
4096/4096 [==============================] - 1s 182us/step - loss: 1.1827 - accuracy: 0.6453
Epoch 2/4
4096/4096 [==============================] - 1s 201us/step - loss: 0.8911 - accuracy: 0.7251
Epoch 3/4
4096/4096 [==============================] - 1s 214us/step - loss: 0.7282 - accuracy: 0.7742
Epoch 4/4
4096/4096 [==============================] - 1s 201us/step - loss: 0.6183 - accuracy: 0.8037
Epoch 1/4
4096/4096 [==============================] - 1s 209us/step - loss: 1.1492 - accuracy: 0.6650
Epoch 2/4
4096/4096 [==============================] - 1s 221us/step - loss: 0.8636 - accuracy: 0.7307
Epoch 3/4
4096/4096 [==============================] - 1s 220us/step - loss: 0.7162 - accuracy: 0.7788
Epoch 4/4
4096/4096 [==============================] - 1s 211us/step - loss: 0.6301 - accuracy: 0.8027
Epoch 1/4
4096/4096 [==============================] - 1s 210us/step - loss: 1.1366 - accuracy: 0.6650
Epoch 2/4
4096/4096 [==============================] - 1s 211us/step - lo

4096/4096 [==============================] - 1s 203us/step - loss: 0.5597 - accuracy: 0.8284
Epoch 1/4
4096/4096 [==============================] - 1s 213us/step - loss: 1.0898 - accuracy: 0.6672
Epoch 2/4
4096/4096 [==============================] - 1s 191us/step - loss: 0.8028 - accuracy: 0.7500
Epoch 3/4
4096/4096 [==============================] - 1s 189us/step - loss: 0.6541 - accuracy: 0.7927
Epoch 4/4
4096/4096 [==============================] - 1s 188us/step - loss: 0.5670 - accuracy: 0.8225
Epoch 1/4
4096/4096 [==============================] - 1s 188us/step - loss: 1.0789 - accuracy: 0.6670
Epoch 2/4
4096/4096 [==============================] - 1s 190us/step - loss: 0.7973 - accuracy: 0.7490
Epoch 3/4
4096/4096 [==============================] - 1s 188us/step - loss: 0.6406 - accuracy: 0.7944
Epoch 4/4
4096/4096 [==============================] - 1s 203us/step - loss: 0.5604 - accuracy: 0.8218
Epoch 1/4
4096/4096 [==============================] - 1s 200us/step - loss: 1.1293

In [92]:
model.save_weights("weights_char_rnn.h5")

In [94]:
seq = "i like"
text = seq + ""
for i in range(700):
    enc_seq = util.one_hot_encode_sequence(seq, char2idx).reshape(1, SEQUENCE_LENGTH, vocab_size)
    preds = model.predict(enc_seq)
    next_char = idx2char[util.sample(preds)]
    text += next_char
    seq = seq[1:]
    seq += next_char
print(text)

i like a star  
To fill the now that sick I will never the night  
And the firapisey  
It's gonna find the sun is the derd in the dry  
  
Enele toonn everything on the pratting aby tonel wey how I well lay love me will the day  
  
Just as look  
Looking down  
The amriting for tros  
sny that ald the  
Stinr you  
So evoe this dingas same  
  
Just as longhr now the rock'n Roller  
And the sosdy I chalfam  
  
(Pick a bale a day  
  
Sometom the head to gan  
The are the dancing Queen, diding how I'n still foreoun  
You peap fert you go love of ours and freet, night  
  
As I surr is froany  
To the bus  
The sing  
One doo  
(Therw in tog the goonn got to me  
  
For and you long tine war  tand


Examples
i like you on my omtowe belnet thing
i like you smertbees the sperid, the

i like  
Lav alllithy ist tive comemongen,  
Bot nrain  moting wipded, plqie tone, hagfue inght ir goint we lingery  
Pookeine mf you of a bit)  
We world yourvew on the Krrowous goneven the titlrus risting in the frid you preaon  
Don't seel) tark nopceer and I had thk rtce smige wnel  
But and eveninge that ceres my momingod aplond thillubn wasto longs gow as sictuplist thum Dild my fingert  sBie one you when I've met at )  
 yaherr I'm just con't kere I bobid you lyokich amw svenaken  
The wey for the nights little and ortroc  
Where is not bitner-belesdd  
Farmar ands to sleas no cachas yours not sol-endr comd flide oh've wlike a jumcabe this lotely to find my wamas aread  
On bofflr whe  
E s

i like Conltare  
Looking for the knaw, goonb folway  
You'll bit's -owing wes down  
(I'  
Uilithe  
In a thannt, ngis lovelight  
come on wosta finky  
Plyinm lowing mepes the nean you spen alonn  
  
Bet a amias-e canot reonef by me in oll fand ther for sther  
Plpaneb and friend -obed be diwan needs 
 of 
  
Nine, pingeen is ther "like the siredrah haning my bindy ond tryet sild 't lound here I  
I dunne  
Ehey bees wnite  
The rock'n roll  
To fell on the durktion  
But in my awning  
Ith  
Every that spanet stige  
Honey honey honey hene  
And belinve  
Sometomerow, thas bidyth-te de,rida, I got to step ty with a  
Long dance  
Swintgres  
You stall  
In't the ny heand to was bock had a fais

i like a star  
To fill the now that sick I will never the night  
And the firapisey  
It's gonna find the sun is the derd in the dry  
  
Enele toonn everything on the pratting aby tonel wey how I well lay love me will the day  
  
Just as look  
Looking down  
The amriting for tros  
sny that ald the  
Stinr you  
So evoe this dingas same  
  
Just as longhr now the rock'n Roller  
And the sosdy I chalfam  
  
(Pick a bale a day  
  
Sometom the head to gan  
The are the dancing Queen, diding how I'n still foreoun  
You peap fert you go love of ours and freet, night  
  
As I surr is froany  
To the bus  
The sing  
One doo  
(Therw in tog the goonn got to me  
  
For and you long tine war  tand